# ノードの停止

---

OpenHPC環境のノードを一時的に停止します。

## VCP の初期化
次のセルを実行すると入力枠が表示されるのでアクセストークンの値を入力してください。

In [ ]:
from getpass import getpass
vcc_access_token = getpass()

入力されたアクセストークンで VCP を初期化します。

In [ ]:
from common import logsetting
from vcpsdk.vcpsdk import VcpSDK
vcp = VcpSDK(vcc_access_token)

エラーになった場合はこの節のセルを全て `unfreeze` してから、もう一度アクセストークンの入力を行ってください。

## 停止する UnitGroup の指定

In [ ]:
ugroup_name = 'TFclusterGPU'

### 操作可能な UnitGroup 名の表示（必要に応じて実行してください）

他の UnitGroup を停止したい場合は、次のセルを実行して UnitGroup 名を参照し、上のセルの ugroup_name を変更して実行してください。

In [ ]:
!ls -1 group_vars/*.yml | sed -e 's/^group_vars\///' -e 's/\.yml//' | sort

## ノードの停止

### UnitGroup の変数の読み込み

UnitGroup の変数を group_vars ファイルから読み込みます。

In [ ]:
%run scripts/group.py
gvars = load_group_vars(ugroup_name)

### ノードの状態確認

現在のノードの状態を確認します。

In [ ]:
ug = vcp.get_ugroup(ugroup_name)
ug.df_nodes()

### 計算ノードの停止

全ての計算ノードを停止します。

In [ ]:
# ★★★ 計算ノード停止：停止確認自動化のためオリジナルを改造 ★★★
unit_compute = ug.get_unit('compute')
unit_compute.power_off_nodes(num_power_off_nodes=gvars['compute_nodes'])

# unit_compute = ug.get_unit('compute')
unit_compute.df_nodes()
if any([node.state != 'POWER_OFF' for node in unit_compute.find_nodes()]):
    raise RuntimeError('ERROR: cannot power off')

### マスターノードの停止

マスターノードを停止します。
> 計算ノードのみを停止したい場合はこの節を実行しないでください。

In [ ]:
# ★★★ 停止、停止確認の自動化 ★★★
unit_master = ug.get_unit('master')
unit_master.power_off_nodes()

# unit_master = ug.get_unit('master')
unit_master.df_nodes()
if any([node.state != 'POWER_OFF' for node in unit_master.find_nodes()]):
    raise RuntimeError('ERROR: cannot power off')